In [ ]:
##### This Python 3 environment comes with many helpful analytics libraries installed
##### It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
##### For example, here's several helpful packages to load in 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
import category_encoders as ce
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.model_selection import KFold
from sklearn import linear_model
from category_encoders import *


##### Input data files are available in the "../input/" directory.
##### For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

##### Any results you write to the current directory are saved as output.

In [ ]:
#read in the files
sample_submission = pd.read_csv("../input/cat-in-the-dat/sample_submission.csv")
test = pd.read_csv("../input/cat-in-the-dat/test.csv")
train = pd.read_csv("../input/cat-in-the-dat/train.csv")

## Exploratory Data Analysis

In [ ]:
print("Shape of training data: {}" .format(train.shape))
train.head()

In [ ]:
train.columns

We can see that the features of the dataset consists of mainly **binary**, **nominal**, **ordinal** and **cyclical**(day,month) features <br>
<br>
**Binary**: Consists of only two unique values. e.g 1/0, True/False, Yes/No <br>
**Nominal**: Values that does not have any ordering <br>
**Ordinal**: Values with specific ordering <br>
**Cyclical**: Values that are recurrent in nature

#### Distribution of Target


In [ ]:
ax = sns.barplot(x='target',y='target', data=train, estimator=lambda x:len(x)/len(train)*100)
ax.set(ylabel ="Frequency in %", title="distribution of target")


### Binary Variables <br>
We will first investigate the distribution of 0s and 1s in the Binary variables

In [ ]:
bin_col = ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4']


grid = gridspec.GridSpec(3, 2)
plt.figure(figsize=(15,20))
for i,col in enumerate(train[bin_col]):
    ax = plt.subplot(grid[i]) 
    sns.countplot(x=col, data=train) 
    ax.set_ylabel('Count')  
    ax.set_xlabel('Values') 
    ax.set_title('{} Distribution'.format(col))
    sizes=[] 
    for p in ax.patches:
        height = p.get_height()
        sizes.append(height)
        ax.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(height/len(train)*100),
                ha="center", fontsize=14) 
    ax.set_ylim(0, max(sizes) * 1.15) 
    
plt.show()

### Preprocessing Binary Variables <br>
Machine Learning algorithm works on numerical data, hence, we will have to map boolean values from *bin_3* and *bin_4* into 0s and 1s

In [ ]:
mapper = {'T': 1, 'F':0, 'Y':1, 'N':0}
#map char into 0/1
train['bin_3'] = train['bin_3'].map(mapper)
test['bin_3'] = test['bin_3'].map(mapper)
train['bin_4'] = train['bin_4'].map(mapper)
test['bin_4'] = test['bin_4'].map(mapper)

In [ ]:
print(train['bin_3'].value_counts())
print(train['bin_4'].value_counts())

### Nominal Variables <br>
We will try to visualize the values and its distribution for the nominal variables

In [ ]:
nom_col = ['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']

for col in nom_col:
    print("Number of unique values for {}: {}".format(col,train[col].nunique()))

We see that *nom_5* to *nom_9* has high cardianality (alot of unique values). For visualisation purposes, we will just look at the nominal features with less than 10 unique values[](http://)

In [ ]:
plt.figure(figsize=(15,20))
grid = gridspec.GridSpec(3, 2)

for i,col in enumerate(train[nom_col[:5]]):
    ax = plt.subplot(grid[i]) 
    sns.countplot(x=col, data=train) 
    ax.set_ylabel('Count')  
    ax.set_xlabel('Values') 
    ax.set_title('{} Distribution'.format(col))
    sizes=[] 
    for p in ax.patches:
        height = p.get_height()
        sizes.append(height)
        ax.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(height/len(train)*100),
                ha="center", fontsize=14) 
    ax.set_ylim(0, max(sizes) * 1.15) 
    
plt.show()

#### One-Hot encoding <br>
We will use one-hot encoding in this case for *nom_0* to *nom_4*. One-hot encoding will create K number os columns based on the unique values and assign a binary value indicating the presence of the features

In [ ]:
def one_hot(df):
    df_work = df[['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4']]
    df_result = pd.get_dummies(df_work, prefix=nom_col[0:5], columns=nom_col[0:5])
    df = pd.concat([df,df_result],axis=1)
    return df

train = one_hot(train)
test = one_hot(test)

In [ ]:
train.columns

#### Nominal Features with high cardianality <br>
Lets look at *nom_5* to *nom_9* values to find out more about the high cardianality features. They are mainly made up of alphanumeic strings that doesnt have any meaning


In this case, i will try to use:
1. **Hashing**
2. **Target Encoding**
3. **Decoding hexademical**

#### Hashing (experiment)

In [ ]:
# for col in nom_col[5:]:
#     col_name = col+'_hash_encode'
#     train[col_name] = train[col].apply(lambda x:hash(str(x)) % 6011)
#     test[col_name] = test[col].apply(lambda x:hash(str(x)) % 6011)

#### Target Encoding

In [ ]:
from category_encoders import TargetEncoder
encoder = TargetEncoder()
for col in nom_col[5:]:
    colname = col+'_target'
    train[colname] = encoder.fit_transform(train['nom_5'], train['target'])
    test[colname] = encoder.transform(test['nom_5'])

#### Decoding Hexadecimal values (experiment)

In [ ]:
# #function to decode hexa to binary
# def hex_to_bin(val):
#     base = 16
#     num_bits = 36
#     res = bin(int(val, base))[2:].zfill(num_bits)
#     return str(res)

# #split bin values into individual columns
# def decode_(df):    
#     for col in nom_col[5:]:
#         df['new_col'] = df[col].apply(lambda x:hex_to_bin(x))
#         df['new_col'] = df['new_col'].apply(lambda x:list(x))
#         new_col = pd.DataFrame(df['new_col'].values.tolist()).rename(columns = lambda x: col+'_decode_'+str(x))
#         df = pd.concat([df,new_col],axis=1)
        
#     return df

# #run for test and train
# train = decode_(train)
# test = decode_(test)

### Ordinal Variable


Visualize the first 3 Ordinal columns

In [ ]:
ord_col = ['ord_0','ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5']
train[ord_col].nunique()  #find out number of distinct categories for each feature

In [ ]:
plt.figure(figsize=(15,20))
grid = gridspec.GridSpec(3, 2)

for i,col in enumerate(train[ord_col[:3]]):
    ax = plt.subplot(grid[i]) 
    sns.countplot(x=col, data=train) 
    ax.set_ylabel('Count')  
    ax.set_xlabel('Values') 
    ax.set_title('{} Distribution'.format(col))
    sizes=[] 
    for p in ax.patches:
        height = p.get_height()
        sizes.append(height)
        ax.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(height/len(train)*100),
                ha="center", fontsize=14) 
    ax.set_ylim(0, max(sizes) * 1.15) 
    
plt.show()

#### Ordinal_0

In [ ]:
train['ord_0'].value_counts()

*ord_0* features are already labeled in an ordinal manner, so we can skip the encoding

 #### Ordinal_1 to Ordinal_4

In [ ]:
print(train['ord_1'].value_counts())
print(train['ord_2'].value_counts())

According to the data, *ord_3* and *ord_4* are made up of ordered ASCII character. So we will encode them into respective ASCII code

In [ ]:
map1 = {'Novice':1, 'Contributor':2, 'Expert':3, 'Master':4, 'Grandmaster':5}
map2 = {'Freezing':1, 'Cold':2, 'Warm':3, 'Hot':4, 'Boiling Hot':5, 'Lava Hot':6}


train['ord_1_encode'] = train['ord_1'].map(map1)
test['ord_1_encode'] = test['ord_1'].map(map1)
train['ord_2_encode'] = train['ord_2'].map(map2)
test['ord_2_encode'] = test['ord_2'].map(map2)
train['ord_3_encode'] = train['ord_3'].apply(lambda x: ord(x))
test['ord_3_encode'] = test['ord_3'].apply(lambda x: ord(x))
train['ord_4_encode'] = train['ord_4'].apply(lambda x: ord(x))
test['ord_4_encode'] = test['ord_4'].apply(lambda x: ord(x))

#### Ordinal_5 <br>



In [ ]:
def freq_encoding(df):
    col_name = 'ord_5'+'_freq_encode'
    fe_train = df.groupby(['ord_5']).size()/len(df)
    df[col_name] = df['ord_5'].map(fe_train)

freq_encoding(train)
freq_encoding(test)

The intuition here is that we see the nature of ordinality in accordance to the assigned ASCII code number..<br>
In this case, we split the 2-char string into individual char, before mapping it into our pre-defined mapper by multiplying the ASCII numbers

In [ ]:
def split(word): 
    lst = [char for char in word]
    return ','.join(lst)

train['ord_5'] = train['ord_5'].apply(lambda x: split(x))
test['ord_5'] = test['ord_5'].apply(lambda x: split(x))

In [ ]:
train[['ord_5_1','ord_5_2']] = train['ord_5'].str.split(',',expand=True)
test[['ord_5_1','ord_5_2']] = test['ord_5'].str.split(',',expand=True)

In [ ]:
train['ord_5_1_encode'] = train['ord_5_1'].apply(lambda x: ord(x))
test['ord_5_1_encode'] = test['ord_5_1'].apply(lambda x: ord(x))

train['ord_5_2_encode'] = train['ord_5_2'].apply(lambda x: ord(x))
test['ord_5_2_encode'] = test['ord_5_2'].apply(lambda x: ord(x))


One-hot encoding (experiment)

In [ ]:
# train = pd.get_dummies(train, prefix=['ord_5_1','ord_5_2'], columns=['ord_5_1','ord_5_2'],sparse=True)   #one-hot apparently works better than using cyclical encoding
# test = pd.get_dummies(test, prefix=['ord_5_1','ord_5_2'], columns=['ord_5_1','ord_5_2'],sparse=True)

### One-Hot Encoding for Time <br>


In [ ]:
train = pd.get_dummies(train, prefix=['day','month'], columns=['day','month'],sparse=True)   #one-hot apparently works better than using cyclical encoding
test = pd.get_dummies(test, prefix=['day','month'], columns=['day','month'],sparse=True)

Cyclic Encoding (experiment)

In [ ]:
# train['day_encode_sin'] = np.sin(2 * np.pi * train['day']/7.0)
# train['day_encode_cos'] = np.cos(2 * np.pi * train['day']/7.0)
# train['month_encode_sin'] = np.sin(2 * np.pi * train['month']/12.0)
# train['month_encode_cos'] = np.cos(2 * np.pi * train['month']/12.0)

# test['day_encode_sin'] = np.sin(2 * np.pi * test['day']/7.0)
# test['day_encode_cos'] = np.cos(2 * np.pi * test['day']/7.0)
# test['month_encode_sin'] = np.sin(2 * np.pi * test['month']/12.0)
# test['month_encode_cos'] = np.cos(2 * np.pi * test['month']/12.0)

In [ ]:
train.columns

#### Drop useless Columns


In [ ]:
#Drop unencoded variables
testid = test['id']
train.drop(['id','bin_0','ord_1','ord_2','ord_3','ord_4','nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4','ord_5_1','ord_5_2'],axis=1,inplace=True)
test.drop(['id','bin_0','ord_1','ord_2','ord_3','ord_4','nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4','ord_5_1','ord_5_2'],axis=1,inplace=True)

In [ ]:
print(train.columns)
train.head()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
# Get feature vector and target vector
y = train['target']
X = train.drop(['target'],axis=1)

In [ ]:
#Weight of Evidence Encoding
enc = WOEEncoder(cols=['ord_5', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']).fit(X, y)
X = enc.transform(X)
test = enc.transform(test)

### Standardisation

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scale_col = ['ord_3_encode','ord_4_encode', 'ord_5_freq_encode','ord_5', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']
scaler = MinMaxScaler()
scaler.fit(X[scale_col])


X[scale_col] = scaler.transform(X[scale_col])
test[scale_col] = scaler.transform(test[scale_col] )

> ## Model Training

In [ ]:
# Split train to train validation set
trainX_init, testX, trainy_init, testy = train_test_split(X, y, test_size=0.2, random_state=2020)

#### SMOTE Oversampling our minority class (1)

In [ ]:
from imblearn.over_sampling import SMOTE
trainX, trainy = SMOTE(random_state = 2020).fit_resample(trainX_init, trainy_init)

In [ ]:
before_smote = pd.DataFrame(trainy_init, columns=['target'])
after_smote = pd.DataFrame(trainy, columns=['target'])

#### Plot the distribution of Target 'Before' and 'After' SMOTE 

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(20,8),ncols=2, nrows=1)
ax1.set_title('Before SMOTE')
ax2.set_title('After SMOTE')
sns.barplot(x='target',y='target', data=before_smote, estimator=lambda x:len(x)/len(before_smote)*100,ax=ax1)
sns.barplot(x='target',y='target', data=after_smote, estimator=lambda x:len(x)/len(after_smote)*100,ax=ax2)
ax1.set_ylabel('Percentage %')
ax2.set_ylabel('Percentage %')
plt.show()

In [ ]:
trainy.value_counts()

In [ ]:
print(trainX.shape)
print(trainy.shape)
print(testX.shape)
print(testy.shape)

### Logistic Regression <br>
Apparently performs on-par or sometimes even better than other complex models


In [ ]:
glm = linear_model.LogisticRegression(C=0.1,solver="liblinear", penalty='l2',random_state=2020) 
glm.fit(trainX, trainy)

In [ ]:
y_pred_lr=glm.predict_proba(trainX)
roc_auc_score(trainy, y_pred_lr[:,1])

In [ ]:
y_pred_lr=glm.predict_proba(testX)
roc_auc_score(testy, y_pred_lr[:,1])

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix

disp = plot_confusion_matrix(glm, testX, testy,
                            cmap=plt.cm.Blues,
                            normalize='true')

y_preds = glm.predict(testX)

print(classification_report(testy, y_preds))


In [ ]:
sub = glm.predict_proba(test)
submission = pd.DataFrame({'id': testid, 'target': sub[:,1]})
submission.to_csv('submission.csv', index=False)